In [13]:
import torch

from datetime import datetime
from pprint import pprint

from config import *
from data.extract import DetectorDataProvider, LookUpTable
from data.util import count_points_between

%reload_ext autoreload
%autoreload 2

In [14]:
READ_START_DATE = datetime.strptime(CONFIG['READ_START_DATE'], DATE_FORMAT)
READ_END_DATE = datetime.strptime(CONFIG['READ_END_DATE'], DATE_FORMAT)

### Read data from the data set

Check list of all sections

In [23]:
sections = lut.get_sections()
pprint(list(sections)[:10])
print(len(sections))

[('4140', '4252'),
 ('5051', 'Brodmühlweg'),
 ('19', '7040'),
 ('8505', 'Am Sportpark'),
 ('4090', '4100'),
 ('5050', '5051'),
 ('14', '4140'),
 ('3150', '3160'),
 ('8401', 'lngolstadter Strasse'),
 ('4210', '4230')]
236


Extract list of detectors for each section

In [4]:
int_det = lut.get_detectors_per_section()
int_det

,Start,End,Detectors
0,4140,4252,[]
1,4252,4140,"[12(DD1), 13(DD2), 14(DD3), 15(DD4)]"
2,5051,Brodmühlweg,[]
3,Brodmühlweg,5051,"[3(DB1), 4(DB2)]"
4,19,7040,[5(DC1)]
...,...,...,...
467,6042,54,[]
468,5020,5030,"[9(DF1), 10(DF2)]"
469,5030,5020,"[1(DA1), 2(DA2), 3(DA3)]"
470,23,4100,"[8(DD1), 9(DD2)]"


Extract counts for all sections

In [5]:
ddp = DetectorDataProvider('Data')

int_det['Counts'] = int_det.apply(lambda sec: ddp.get_counts_entering_section(sec['End'], sec['Detectors'], READ_START_DATE, READ_END_DATE), axis=1)
int_det

,Start,End,Detectors,Counts
0,4140,4252,[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,4252,4140,"[12(DD1), 13(DD2), 14(DD3), 15(DD4)]","[1.0, 0.0, 0.0, 0.0, 2.0, 3.0, 0.0, 0.0, 2.0, ..."
2,5051,Brodmühlweg,[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,Brodmühlweg,5051,"[3(DB1), 4(DB2)]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
4,19,7040,[5(DC1)],"[0.0, 1.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 1.0, ..."
...,...,...,...,...
467,6042,54,[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
468,5020,5030,"[9(DF1), 10(DF2)]","[0.0, 2.0, 3.0, 3.0, 5.0, 1.0, 3.0, 2.0, 2.0, ..."
469,5030,5020,"[1(DA1), 2(DA2), 3(DA3)]","[4.0, 6.0, 8.0, 7.0, 6.0, 0.0, 3.0, 5.0, 1.0, ..."
470,23,4100,"[8(DD1), 9(DD2)]","[0.0, 1.0, 4.0, 2.0, 0.0, 2.0, 1.0, 1.0, 0.0, ..."


### Write data into a matrix

Write counts into a matrix
$$Q=[q_1,q_2,...,q_p]$$
for
$$q_i=\set{z(s_i,t_1),z(s_i,t_2),...z(s_i,t_d)}^T,$$
$z(s_i,t_j)$ is the traffic flow of the road section $s_i$ within the time interval $(t_0,t_j)$

In [26]:
mat_q = torch.tensor(int_det['Counts'].tolist()).T
torch.save(mat_q, out_path('mat_q.pt'))
assert mat_q.shape == (count_points_between(READ_START_DATE, READ_END_DATE), 2 * len(sections))
mat_q.shape                                                         

torch.Size([32064, 472])